In [1]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [2]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import os

# 데이터 확인


In [3]:
path="/gdrive/My Drive/Colab Notebooks/dacon/credit_card"
csv=pd.read_csv(path+"/train.csv")
csv.describe()

,index,child_num,income_total,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,family_size,begin_month,credit
count,26457.000000,26457.000000,2.645700e+04,26457.000000,26457.000000,26457.0,26457.000000,26457.000000,26457.000000,26457.000000,26457.000000,26457.000000
mean,13228.000000,0.428658,1.873065e+05,-15958.053899,59068.750728,1.0,0.224742,0.294251,0.091280,2.196848,-26.123294,1.519560
std,7637.622372,0.747326,1.018784e+05,4201.589022,137475.427503,0.0,0.417420,0.455714,0.288013,0.916717,16.559550,0.702283
min,0.000000,0.000000,2.700000e+04,-25152.000000,-15713.000000,1.0,0.000000,0.000000,0.000000,1.000000,-60.000000,0.000000
25%,6614.000000,0.000000,1.215000e+05,-19431.000000,-3153.000000,1.0,0.000000,0.000000,0.000000,2.000000,-39.000000,1.000000
50%,13228.000000,0.000000,1.575000e+05,-15547.000000,-1539.000000,1.0,0.000000,0.000000,0.000000,2.000000,-24.000000,2.000000
75%,19842.000000,1.000000,2.250000e+05,-12446.000000,-407.000000,1.0,0.000000,1.000000,0.000000,3.000000,-12.000000,2.000000
max,26456.000000,19.000000,1.575000e+06,-7705.000000,365243.000000,1.0,1.000000,1.000000,1.000000,20.000000,0.000000,2.000000


In [4]:
csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26457 entries, 0 to 26456
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   index          26457 non-null  int64  
 1   gender         26457 non-null  object 
 2   car            26457 non-null  object 
 3   reality        26457 non-null  object 
 4   child_num      26457 non-null  int64  
 5   income_total   26457 non-null  float64
 6   income_type    26457 non-null  object 
 7   edu_type       26457 non-null  object 
 8   family_type    26457 non-null  object 
 9   house_type     26457 non-null  object 
 10  DAYS_BIRTH     26457 non-null  int64  
 11  DAYS_EMPLOYED  26457 non-null  int64  
 12  FLAG_MOBIL     26457 non-null  int64  
 13  work_phone     26457 non-null  int64  
 14  phone          26457 non-null  int64  
 15  email          26457 non-null  int64  
 16  occyp_type     18286 non-null  object 
 17  family_size    26457 non-null  float64
 18  begin_

# preprocess

In [5]:
class Preprocess_class:
    def __init__(self,path):
        print("---------전처리를 시작합니다----------")
        self.my_csv=pd.read_csv(path)
        self.size=len(self.my_csv) # 데이터개수
        self.columns=self.my_csv.columns # 모든칼럼

        # 명목척도와 그외로 나누기 
        self.nominal=["gender","car","reality","income_type","edu_type","family_type","house_type","work_phone","phone","email","occyp_type"] # FLAG_MOBIL은 모두 1이므로 제외
        self.notNominal=["income_total","child_num","DAYS_BIRTH","DAYS_EMPLOYED","family_size","begin_month"]

        self.df=pd.DataFrame()
        self.df_y=pd.DataFrame()
        
        self.x=[]
        self.y=[]

    def getX(self):
        return self.x

    def getY(self):
        return self.y

    #------------------ x, y preprocesses------------------
    def pick_preprocessing_x(self,preprocessNum):
        if preprocessNum==1:
            self.preprocess_1_onehot()
        elif preprocessNum==2:
            self.preprocess_2_delCol()
        elif preprocessNum==3:
            self.preprocess_3_factorizing()
        elif preprocessNum==4:
            self.preprocess_1_onehot()
        elif preprocessNum==5:
            self.preprocess_4_bitmask()
        
    def pick_preproceesing_y(self,preprocessNum):
        if preprocessNum==1:
            self.preprocessing_y_dummy()
        elif preprocessNum==2:
            self.preprocessing_y_dummy()
        elif preprocessNum==3:
            self.preprocessing_y_dummy()
        elif preprocessNum==4:
            self.preprocessing_y_numbering()
        elif preprocessNum==5:
            self.preprocessing_y_numbering()
            
    #----------------types of preprocesses_x------------------
    def preprocess_1_onehot(self):
        self.nominal_onehot() # 명목척도 전처리
        self.notNominal_scaling() # 비명목척도 전처리

        self.transformation(self.df,self.x)
        self.x=np.array(self.x)
        
    def preprocess_2_delCol(self):
        delCol=["work_phone","phone","email"]
        for col in delCol:
            if col in self.nominal:
                self.nominal.remove(col)
            else:
                self.notNominal.remove(col)

        self.nominal_onehot() # 명목척도 전처리
        self.notNominal_scaling() # 비명목척도 전처리

        self.transformation(self.df,self.x)
        self.x=np.array(self.x)

    def preprocess_3_factorizing(self):
        self.nominal_factorizing()
        self.notNominal_scaling()
        
        self.transformation(self.df,self.x)
        self.x=np.array(self.x)

    def preprocess_4_bitmask(self):
        self.nominal_factorizing()
        self.bitmasking()

        self.notNominal_scaling()
        
        display(self.df)
        
        self.transformation(self.df,self.x)
        self.x=np.array(self.x)


    #----------------types of preprocesses_y------------------
    def preprocessing_y_dummy(self):
        self.df_y=pd.get_dummies(self.my_csv["credit"])
        self.df_y.columns=[0,1,2]
        self.transformation(self.df_y,self.y)
        self.y=np.array(self.y)

    def preprocessing_y_numbering(self):
        self.y=list(self.my_csv["credit"])
        self.y=np.array(list(map(str,self.y)))

        # L=[]
        # for i in range(self.size):
        #     L.append([self.my_csv["credit"][i]])
        # # self.y=np.array([self.my_csv["credit"]])
        # self.y=np.array(L)

    #--------------funcs------------------
    def bitmasking(self):
        uniqD={}
        for col in self.nominal:
            uniq=len(self.df[col].unique())
            print(uniq,col)
            if uniq in uniqD:
                uniqD[uniq].append(col)
            else:
                uniqD[uniq]=[col]
        print(uniqD)
        
        newDataFrame=pd.DataFrame()
        for domainNum in uniqD:
            tempD={domainNum:[]}
            for i in range(self.size):
                a=0
                for j,col in enumerate(uniqD[domainNum]):
                    a+=self.df[col][i]*(domainNum**(j))
                tempD[domainNum].append(a)

            tempD=pd.DataFrame(tempD)
            newDataFrame=pd.concat([newDataFrame,tempD],axis=1)

        self.df=newDataFrame
            


    def nominal_factorizing(self):
        for col in self.nominal:
            colDataFrame=pd.DataFrame(self.my_csv[col].factorize()[0],columns=[col])
            self.df=pd.concat([self.df,colDataFrame],axis=1)
            

    def nominal_numbering(self):
        for col in self.nominal:
            uniques=self.my_csv[col].unique()
            uniqueSize=len(uniques)
            d={}

            for i in range(uniqueSize):
                d[uniques[i]]=i
    
            for i in range(self.size):
                self.my_csv[col][i]=int(d[self.my_csv[col][i]])

            self.df=pd.concat([self.df,self.my_csv[col]],axis=1)
            self.df=self.df.astype({col:"int"})

    def nominal_onehot(self):
        # make nominal onehot
        for col in self.nominal:
            addCol=pd.get_dummies(self.my_csv[col])
            self.df=pd.concat([self.df,addCol],axis=1)

    def notNominal_scaling(self):
        # 양수로 바꾸기
        for i in range(self.size):
            self.my_csv["DAYS_BIRTH"][i]=abs(self.my_csv["DAYS_BIRTH"][i])

        for i in range(self.size):
            if self.my_csv["DAYS_EMPLOYED"][i]>0: 
                self.my_csv["DAYS_EMPLOYED"][i]=0
            else:
                self.my_csv["DAYS_EMPLOYED"][i]=abs(self.my_csv["DAYS_EMPLOYED"][i])
                
        # 스케일링
        for col in self.notNominal:
            addCol=pd.DataFrame(self.my_scaling(self.my_csv[col]))
            addCol.columns=[col]
            self.df=pd.concat([self.df,addCol],axis=1)

    # df -> np array
    def transformation(self,df,L):
        for i in range(self.size):
            temp_x=np.array(df.iloc[i])
            L.append(temp_x)

    # 스케일링
    def my_scaling(self,scaling_col):
        scaling_col=np.array(scaling_col).reshape(-1,1)
        
        scaler = preprocessing.MinMaxScaler()
        scaler_fit = scaler.fit(scaling_col)
        result=scaler_fit.transform(scaling_col)

        return result


In [6]:
# "DAYS_BIRTH" 
def days_birth_toYear(): # 년(나이)로 바꾸기
    for i in range(size):
        csv["DAYS_BIRTH"][i]=((-1)*csv["DAYS_BIRTH"][i])//365
    display(csv["DAYS_BIRTH"])

# "DAYS_EMPLOYED"
def days_employed_toMonth(): # 달로 바꾸기
    for i in range(size):
        if csv["DAYS_EMPLOYED"][i]>0:
            csv["DAYS_EMPLOYED"][i]=0
        else:
            csv["DAYS_EMPLOYED"][i]=(abs(csv["DAYS_EMPLOYED"][i]))//30
    display(csv["DAYS_EMPLOYED"])

# modeling

In [7]:
import tensorflow as tf
from tensorflow.keras import layers  

# bagging
from sklearn import tree
from sklearn.tree import export_graphviz
from sklearn.ensemble import RandomForestClassifier

# boosting
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier

# regression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier

import matplotlib.pyplot as plt
import pydotplus
from IPython.display import Image

import pickle



In [8]:
class Model_class:
    def __init__(self,path,x_train,y_train,subNum):
        print("---------모델학습을 시작합니다----------")

        self.colSize=len(x_train[0])

        try:
            self.outputSize=len(y_train[0]) #dummy형
        except:
            self.outputSize=1 #숫자형

        self.x_train=x_train
        self.y_train=y_train
        self.path=path
        self.modelNum=0
        self.subNum=subNum
        #self.model,self.hist

    def model_pick(self,modelNum):
        if modelNum==1:
            self.model_1_dnn()
        elif modelNum==2:
            self.model_2_dnn()
        elif modelNum==3:
            self.model_3_tree()
        elif modelNum==4:
            self.model_4_randomForest()
        elif modelNum==5:
            self.model_5_lightGBM()
        elif modelNum==6:
            self.model_6_XGBClassifier()
        elif modelNum==7:
            self.model_7_logicticRegression()
        elif modelNum==8:
            self.model_8_GradientBoostingClassifier()
            


    def model_1_dnn(self):
        my_epochs=3000
        # ---------------모델 구조 생성---------------
        self.model = tf.keras.Sequential()  
        # input node:4, hidden node:25
        self.model.add(layers.Dense(25, input_shape=(self.colSize,)))  
        self.model.add(layers.Activation('relu'))  
        self.model.add(layers.Dropout(0.2))        
        # hidden node : 25
        self.model.add(layers.Dense(25))       
        self.model.add(layers.Activation('relu'))
        self.model.add(layers.Dropout(0.2))
        # output node : 3
        self.model.add(layers.Dense(self.outputSize))
        self.model.add(layers.Activation('softmax')) 
        
        # ---------------모델 구축하기---------------
        self.model.compile(
            loss='categorical_crossentropy',  #다중 교차엔트로피
            optimizer="rmsprop",   #최적화 기법 중 하나
            metrics=['accuracy'])  #정확도 측정

        # ---------------콜백 ---------------
        folder_directory=self.path+f"/models/{self.subNum}"

        # 폴더있고 모델도있으면
        if os.path.exists(folder_directory) and os.listdir(folder_directory):
            print("이어서 학습진행")
            checkPoint_dir = os.path.dirname(folder_directory+"/model_{epoch}.ckpt")
            latest = tf.train.latest_checkpoint(checkPoint_dir)
            self.model.load_weights(latest)                
        else:
            print("새로운 학습 시작")
            # 폴더도 없으면 폴더 생성 
            if not os.path.exists(folder_directory):
                os.makedirs(folder_directory) # 폴더만들기

        checkPoint_path = folder_directory+"/model_{epoch}.ckpt"
        cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkPoint_path,
                                                    save_weights_only=True,
                                                    verbose=1,
                                                    period=3) 
        earlyStopping_callback=tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=200)
    
        self.hist = self.model.fit(self.x_train, self.y_train, batch_size=30, epochs=3000, validation_split=0.2, 
                        callbacks=[cp_callback,earlyStopping_callback],  verbose=1)

        self.visualize_dnn()

    def model_2_dnn(self):
        # ---------------모델 구조 생성---------------
        self.model = tf.keras.Sequential()  
        # input node:4, hidden node:50
        self.model.add(layers.Dense(50, input_shape=(self.colSize,)))  
        self.model.add(layers.Activation('relu'))  
        self.model.add(layers.Dropout(0.2))        
        # hidden node : 25
        self.model.add(layers.Dense(25))       
        self.model.add(layers.Activation('relu'))
        self.model.add(layers.Dropout(0.2))
        # hidden node : 25
        self.model.add(layers.Dense(25))       
        self.model.add(layers.Activation('relu'))
        self.model.add(layers.Dropout(0.2))
        # output node : 3
        self.model.add(layers.Dense(self.outputSize))
        self.model.add(layers.Activation('softmax')) 
        
        # ---------------모델 구축하기---------------
        self.model.compile(
            loss='categorical_crossentropy',  #다중 교차엔트로피
            optimizer="rmsprop",   #최적화 기법 중 하나
            metrics=['accuracy'])  #정확도 측정

        # ---------------콜백 ---------------
        folder_directory=self.path+f"/models/{self.subNum}"

        # 폴더있고 모델도있으면
        if os.path.exists(folder_directory) and os.listdir(folder_directory):
            print("이어서 학습진행")
            checkPoint_dir = os.path.dirname(folder_directory+"/model_{epoch}.ckpt")
            latest = tf.train.latest_checkpoint(checkPoint_dir)
            self.model.load_weights(latest)                
        else:
            print("새로운 학습 시작")
            # 폴더도 없으면 폴더 생성 
            if not os.path.exists(folder_directory):
                os.makedirs(folder_directory) # 폴더만들기

        checkPoint_path = folder_directory+"/model_{epoch}.ckpt"
        cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkPoint_path,
                                                    save_weights_only=True,
                                                    verbose=1,
                                                    period=3) 
        earlyStopping_callback=tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=200)

        self.hist = self.model.fit(self.x_train, self.y_train, batch_size=10, epochs=3000, validation_split=0.3, 
                        callbacks=[cp_callback,earlyStopping_callback],  verbose=1) 

        self.visualize_dnn()

    def model_3_tree(self):
        self.model = tree.DecisionTreeClassifier(criterion='gini', max_depth=6, random_state=0)
        print(self.x_train)
        print(self.y_train)
        self.model.fit(self.x_train, self.y_train)

        self.visualize_tree()

    def model_4_randomForest(self):
        self.model=RandomForestClassifier()
        self.model.fit(self.x_train,self.y_train)

    def model_5_lightGBM(self):
        self.model=LGBMClassifier(learning_rate=0.05, n_estimators=100, random_state=123)
        self.model.fit(self.x_train, self.y_train)

    def model_6_XGBClassifier(self):
        self.model=XGBClassifier(learning_rate=0.05, n_estimators=100, random_state=123)
        self.model.fit(self.x_train, self.y_train)

    def model_7_logicticRegression(self):
        self.model=LogisticRegression()
        self.model.fit(self.x_train, self.y_train)

    def model_8_GradientBoostingClassifier(self):
        self.model=GradientBoostingClassifier()
        self.model.fit(self.x_train, self.y_train)

    def visualize_dnn(self):
        fig, loss_ax = plt.subplots()

        acc_ax = loss_ax.twinx()

        loss_ax.plot(self.hist.history['loss'], 'y', label='train loss')
        loss_ax.plot(self.hist.history['val_loss'], 'r', label='val loss')

        acc_ax.plot(self.hist.history['accuracy'], 'b', label='train acc')
        acc_ax.plot(self.hist.history['val_accuracy'], 'g', label='val acc')

        loss_ax.set_xlabel('epoch')
        loss_ax.set_ylabel('loss')
        acc_ax.set_ylabel('accuray')

        loss_ax.legend(loc='upper left')
        acc_ax.legend(loc='lower left')

        plt.show()

    def visualize_tree(self):
        tree.plot_tree(self.model) 

    def my_model_save(self):
        with open(self.path+f"/models/model_{self.subNum}.pickle",'wb') as fw:
            pickle.dump(self.model, fw)

        # self.model.save(self.path+f"/models/model_{self.subNum}.h5")

In [10]:
print('''
6 : dnn
7 : decision tree
8 : randomforest
9 : lightGBM
10 : XGBoost
11 : bitmast,xgboost
12 : logisticLinear
13 : gradient boosting
''')

finalModel=int(input("최종모델 번호를 입력히시오 : "))

if finalModel==6:
    preprocessNum,modelNum=3,2
elif finalModel==7:
    preprocessNum,modelNum=4,3
elif finalModel==8:
    preprocessNum,modelNum=4,4
elif finalModel==9:
    preprocessNum,modelNum=4,5
elif finalModel==10:
    preprocessNum,modelNum=4,6
elif finalModel==11:
    preprocessNum,modelNum=5,6
elif finalModel==12:
    preprocessNum,modelNum=4,7
elif finalModel==13:
    preprocessNum,modelNum=4,8


# 전처리
preprocess_object=Preprocess_class(path+"/train.csv")
preprocess_object.pick_preprocessing_x(preprocessNum)
preprocess_object.pick_preproceesing_y(preprocessNum)

x_train=preprocess_object.getX()
y_train=preprocess_object.getY()

display(x_train)
display(y_train)

# 모델링
model_object=Model_class(path,x_train,y_train,finalModel)
model_object.model_pick(modelNum)

model_object.my_model_save()


6 : dnn
7 : decision tree
8 : randomforest
9 : lightGBM
10 : XGBoost
11 : bitmast,xgboost
12 : logisticLinear
13 : gradient boosting

최종모델 번호를 입력히시오 : 13
---------전처리를 시작합니다----------


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:165: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:171: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


array([[1.        , 0.        , 1.        , ..., 0.29968816, 0.05263158,
        0.9       ],
       [1.        , 0.        , 1.        , ..., 0.09800802, 0.10526316,
        0.91666667],
       [0.        , 1.        , 0.        , ..., 0.28218672, 0.05263158,
        0.63333333],
       ...,
       [1.        , 0.        , 0.        , ..., 0.12823776, 0.05263158,
        0.58333333],
       [0.        , 1.        , 1.        , ..., 0.00680965, 0.        ,
        0.01666667],
       [1.        , 0.        , 1.        , ..., 0.06446891, 0.05263158,
        0.85      ]])

array(['1.0', '1.0', '2.0', ..., '2.0', '2.0', '2.0'], dtype='<U3')

---------모델학습을 시작합니다----------
